In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
pd.set_option('display.max_columns', None)
traindata=pd.read_csv(r"C:\Users\Anindita\Desktop\Codsoft\credit card fraud\fraudTrain.csv")

In [3]:
traindata.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,01-01-2019 00:00,2.700000e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",09-03-1988,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,01-01-2019 00:00,6.300000e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,21-06-1978,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,01-01-2019 00:00,3.890000e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,19-01-1962,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,01-01-2019 00:01,3.530000e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,12-01-1967,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,01-01-2019 00:03,3.760000e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,28-03-1986,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [4]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1048575 non-null  int64  
 1   trans_date_trans_time  1048575 non-null  object 
 2   cc_num                 1048575 non-null  float64
 3   merchant               1048575 non-null  object 
 4   category               1048575 non-null  object 
 5   amt                    1048575 non-null  float64
 6   first                  1048575 non-null  object 
 7   last                   1048575 non-null  object 
 8   gender                 1048575 non-null  object 
 9   street                 1048575 non-null  object 
 10  city                   1048575 non-null  object 
 11  state                  1048575 non-null  object 
 12  zip                    1048575 non-null  int64  
 13  lat                    1048575 non-null  float64
 14  long              

In [5]:
traindata.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [6]:
traindata['is_fraud'].value_counts()

0    1042569
1       6006
Name: is_fraud, dtype: int64

In [7]:
#0--->legitimate
#1---->fraud
legit_train=traindata[traindata.is_fraud==0]
fraud_train=traindata[traindata.is_fraud==1]

In [8]:
print(legit_train.shape)
print(fraud_train.shape)

(1042569, 23)
(6006, 23)


In [9]:
legit_train.amt.describe()

count    1.042569e+06
mean     6.762744e+01
std      1.536956e+02
min      1.000000e+00
25%      9.600000e+00
50%      4.722000e+01
75%      8.247000e+01
max      2.894890e+04
Name: amt, dtype: float64

In [10]:
fraud_train.amt.describe()

count    6006.000000
mean      530.573492
std       391.333069
min         1.180000
25%       241.577500
50%       391.165000
75%       901.950000
max      1371.810000
Name: amt, dtype: float64

In [11]:
traindata.groupby('is_fraud').mean()

C:\Users\Anindita\AppData\Local\Temp\ipykernel_18984\3676514393.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  traindata.groupby('is_fraud').mean()


,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,
0,524494.707643,4.174085e+17,67.627445,48805.355338,38.532842,-90.228376,89015.900200,1.344913e+09,38.532993,-90.228625
1,488231.463869,3.775093e+17,530.573492,48148.078422,38.623988,-89.858250,96323.951715,1.343602e+09,38.615091,-89.853555


In [12]:
#undersampling
legit_train_sample=legit_train.sample(n=6006)

In [13]:
newtraindata=pd.concat([legit_train_sample,fraud_train],axis=0)

In [14]:
label_encoder = preprocessing.LabelEncoder()
newtraindata['gender']= label_encoder.fit_transform(newtraindata['gender'])
newtraindata['gender'].unique()

array([0, 1])

In [15]:
newtraindata.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
322944,322944,03-06-2019 12:45,3.570000e+15,fraud_O'Keefe-Wisoky,food_dining,10.34,Kristina,Stewart,0,47029 Jimmy Tunnel Apt. 106,Newhall,CA,91321,34.3795,-118.5230,34882,Health physicist,25-04-1971,970432a6ca357036e59f652919e6fa30,1338727515,34.743773,-118.235508,0
829372,829372,10-12-2019 14:14,1.800000e+14,"fraud_Labadie, Treutel and Bode",shopping_net,253.63,Michelle,Woods,0,952 Joseph Throughway,Munith,MI,49259,42.3703,-84.2485,2523,Geophysicist/field seismologist,21-03-1988,c636666bdd0e1a61116dd899d86a88a3,1355148859,42.896145,-83.365972,0
1014689,1014689,22-02-2020 22:23,3.500000e+15,fraud_Zulauf LLC,personal_care,47.97,Kenneth,Sanchez,1,0110 Ashley Forest,Tekoa,WA,99033,47.2271,-117.0819,895,Clothing/textile technologist,31-05-1999,c25f196e1cad33f6773aa0e7d4e2954e,1361571818,46.509501,-117.633657,0
247023,247023,04-05-2019 04:11,4.170000e+15,"fraud_Osinski, Ledner and Leuschke",grocery_pos,69.28,Samuel,Frey,1,830 Myers Plaza Apt. 384,Edmond,OK,73034,35.6665,-97.4798,116001,Media buyer,10-05-1993,60e7f1ff5e129e729b00cc01c8d87bf3,1336104684,34.865921,-97.501636,0
218817,218817,21-04-2019 11:11,3.570000e+15,fraud_Goodwin-Nitzsche,grocery_pos,114.47,Joanne,Williams,0,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",17-01-1990,42c0306283e223f5c3f8907d99e65427,1335006698,39.582586,-110.805485,0


In [16]:
newtraindata['is_fraud'].value_counts()

0    6006
1    6006
Name: is_fraud, dtype: int64

In [17]:
newtraindata.groupby('is_fraud').mean()

C:\Users\Anindita\AppData\Local\Temp\ipykernel_18984\3596457971.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  newtraindata.groupby('is_fraud').mean()


,Unnamed: 0,cc_num,amt,gender,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
is_fraud,,,,,,,,,,,
0,522329.335831,4.047333e+17,67.593923,0.469031,48110.884782,38.478956,-89.745361,89067.173327,1.344835e+09,38.491276,-89.751029
1,488231.463869,3.775093e+17,530.573492,0.504662,48148.078422,38.623988,-89.858250,96323.951715,1.343602e+09,38.615091,-89.853555


In [18]:
newtraindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12012 entries, 322944 to 1047918
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12012 non-null  int64  
 1   trans_date_trans_time  12012 non-null  object 
 2   cc_num                 12012 non-null  float64
 3   merchant               12012 non-null  object 
 4   category               12012 non-null  object 
 5   amt                    12012 non-null  float64
 6   first                  12012 non-null  object 
 7   last                   12012 non-null  object 
 8   gender                 12012 non-null  int32  
 9   street                 12012 non-null  object 
 10  city                   12012 non-null  object 
 11  state                  12012 non-null  object 
 12  zip                    12012 non-null  int64  
 13  lat                    12012 non-null  float64
 14  long                   12012 non-null  float64


In [19]:
X_traindata=newtraindata.drop(columns=['is_fraud','trans_date_trans_time','merchant','category','first','last','street','city','state','job','dob','trans_num'],axis=1)
Y_traindata=newtraindata['is_fraud']

In [20]:
print(X_traindata)

         Unnamed: 0        cc_num     amt  gender    zip      lat      long  \
322944       322944  3.570000e+15   10.34       0  91321  34.3795 -118.5230   
829372       829372  1.800000e+14  253.63       0  49259  42.3703  -84.2485   
1014689     1014689  3.500000e+15   47.97       1  99033  47.2271 -117.0819   
247023       247023  4.170000e+15   69.28       1  73034  35.6665  -97.4798   
218817       218817  3.570000e+15  114.47       0  84002  40.3207 -110.4360   
...             ...           ...     ...     ...    ...      ...       ...   
1047089     1047089  3.590000e+15  690.49       0  57374  43.7557  -97.5936   
1047157     1047157  3.550000e+15  324.74       1  76008  32.7004  -97.6039   
1047208     1047208  3.590000e+15  331.33       0  57374  43.7557  -97.5936   
1047521     1047521  3.590000e+15  356.20       0  57374  43.7557  -97.5936   
1047918     1047918  3.590000e+15  249.56       0  57374  43.7557  -97.5936   

         city_pop   unix_time  merch_lat  merch_lon

In [21]:
print(Y_traindata)

322944     0
829372     0
1014689    0
247023     0
218817     0
          ..
1047089    1
1047157    1
1047208    1
1047521    1
1047918    1
Name: is_fraud, Length: 12012, dtype: int64


In [22]:
X_train=X_traindata

In [23]:
Y_train=Y_traindata

In [24]:
testdata=pd.read_csv(r'C:\Users\Anindita\Desktop\Codsoft\credit card fraud\fraudTest.csv')

In [25]:
testdata.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,21-06-2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,SC,29209,33.9659,-80.9355,333497,Mechanical engineer,19-03-1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,21-06-2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,UT,84002,40.3207,-110.4360,302,"Sales professional, IT",17-01-1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,21-06-2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,NY,11710,40.6729,-73.5365,34496,"Librarian, public",21-10-1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,21-06-2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,FL,32780,28.5697,-80.8191,54767,Set designer,25-07-1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,21-06-2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,MI,49632,44.2529,-85.0170,1126,Furniture designer,06-07-1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [26]:
testdata['is_fraud'].value_counts()

0    553574
1      2145
Name: is_fraud, dtype: int64

In [27]:
legit_test=testdata[testdata.is_fraud==0]
fraud_test=testdata[testdata.is_fraud==1]
legit_test_sample=legit_test.sample(n=2145)
newtestdata=pd.concat([legit_test_sample,fraud_test],axis=0)
label_encoder = preprocessing.LabelEncoder()
newtestdata['gender']= label_encoder.fit_transform(newtestdata['gender'])
newtestdata['gender'].unique()

array([1, 0])

In [28]:
newtestdata.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
473315,473315,13-12-2020 23:33,4.605900e+12,fraud_Block-Hauck,travel,6.98,Vincent,Mueller,1,79019 Taylor Cliffs,Dallas,TX,75238,32.8770,-96.7080,1263321,Drilling engineer,28-06-1959,abc6b8f1509a0e764a72d6b67910212d,1386977639,32.377303,-95.945246,0
414915,414915,30-11-2020 20:13,6.597890e+15,fraud_Larkin Ltd,kids_pets,56.47,Bill,Lane,1,4725 Hurst Curve Apt. 634,Holliday,TX,76366,33.6751,-98.6576,2317,Chief Technology Officer,23-08-1964,dbb3cea739228d23081489c26b63a4bf,1385842419,34.337711,-97.667283,0
474105,474105,14-12-2020 03:14,3.534090e+15,"fraud_Kovacek, Dibbert and Ondricka",grocery_pos,157.04,Jeremy,White,1,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,12-01-1967,0d0579ff4ab8687921fc4d988084c03c,1386990877,45.468204,-111.290172,0
399045,399045,24-11-2020 16:29,3.595190e+15,fraud_Kuhn Group,food_dining,47.06,Dustin,Roberts,1,3283 James Station,Fayetteville,NC,28314,35.0583,-79.0080,238602,"Scientist, research (maths)",14-05-1993,eb367ea915b01ba414055a60bb5b5db4,1385310585,34.821861,-79.695506,0
20897,20897,28-06-2020 17:35,6.538440e+15,fraud_Corwin-Romaguera,travel,5.66,Gina,Grimes,0,444 Robert Mews,Clarks Mills,PA,16114,41.3851,-80.1752,606,Energy manager,22-09-1997,accc744535651df18da93cba286ddbac,1372440930,42.082340,-79.586500,0


In [29]:
X_testdata=newtestdata.drop(columns=['is_fraud','trans_date_trans_time','merchant','category','first','last','street','city','state','job','dob','trans_num'],axis=1)
Y_testdata=newtestdata['is_fraud']

In [30]:
print(X_testdata)

        Unnamed: 0        cc_num      amt  gender    zip      lat      long  \
473315      473315  4.605900e+12     6.98       1  75238  32.8770  -96.7080   
414915      414915  6.597890e+15    56.47       1  76366  33.6751  -98.6576   
474105      474105  3.534090e+15   157.04       1  59632  46.2306 -112.1138   
399045      399045  3.595190e+15    47.06       1  28314  35.0583  -79.0080   
20897        20897  6.538440e+15     5.66       0  16114  41.3851  -80.1752   
...            ...           ...      ...     ...    ...      ...       ...   
517197      517197  2.242180e+15  1041.51       1  38668  34.6323  -89.8855   
517274      517274  2.242180e+15   868.09       1  38668  34.6323  -89.8855   
517341      517341  2.242180e+15  1039.42       1  38668  34.6323  -89.8855   
517529      517529  2.242180e+15   289.27       1  38668  34.6323  -89.8855   
517571      517571  2.242180e+15   766.38       1  38668  34.6323  -89.8855   

        city_pop   unix_time  merch_lat  merch_long

In [31]:
print(Y_testdata)

473315    0
414915    0
474105    0
399045    0
20897     0
         ..
517197    1
517274    1
517341    1
517529    1
517571    1
Name: is_fraud, Length: 4290, dtype: int64


In [32]:
X_test=X_testdata
Y_test=Y_testdata

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
# Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train, Y_train)

LogisticRegression()

In [35]:
# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, Y_train)

DecisionTreeClassifier()

In [36]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, Y_train)

RandomForestClassifier()

In [37]:
# Logistic Regression
lr_train_predictions = lr_model.predict(X_train)
print("Logistic Regression:")
print("For Training data:")
print("Accuracy:", accuracy_score(Y_train, lr_train_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_train, lr_train_predictions))
print("Classification Report:\n", classification_report(Y_train, lr_train_predictions))

lr_test_predictions = lr_model.predict(X_test)
print("Logistic Regression:")
print("For Testing data:")
print("Accuracy:", accuracy_score(Y_test, lr_test_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_test, lr_test_predictions))
print("Classification Report:\n", classification_report(Y_test, lr_test_predictions))

Logistic Regression:
For Training data:
Accuracy: 0.8545621045621046
Confusion Matrix:
 [[5715  291]
 [1456 4550]]
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.95      0.87      6006
           1       0.94      0.76      0.84      6006

    accuracy                           0.85     12012
   macro avg       0.87      0.85      0.85     12012
weighted avg       0.87      0.85      0.85     12012

Logistic Regression:
For Testing data:
Accuracy: 0.5645687645687646
Confusion Matrix:
 [[2144    1]
 [1867  278]]
Classification Report:
               precision    recall  f1-score   support

           0       0.53      1.00      0.70      2145
           1       1.00      0.13      0.23      2145

    accuracy                           0.56      4290
   macro avg       0.77      0.56      0.46      4290
weighted avg       0.77      0.56      0.46      4290



In [38]:
# Decision Tree
dt_train_predictions = dt_model.predict(X_train)
print("Decision Tree:")
print("For Training data:")
print("Accuracy:", accuracy_score(Y_train, dt_train_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_train, dt_train_predictions))
print("Classification Report:\n", classification_report(Y_train, dt_train_predictions))

dt_test_predictions = dt_model.predict(X_test)
print("\nDecision Tree:")
print("For Testing data:")
print("Accuracy:", accuracy_score(Y_test, dt_test_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_test, dt_test_predictions))
print("Classification Report:\n", classification_report(Y_test, dt_test_predictions))


Decision Tree:
For Training data:
Accuracy: 1.0
Confusion Matrix:
 [[6006    0]
 [   0 6006]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6006
           1       1.00      1.00      1.00      6006

    accuracy                           1.00     12012
   macro avg       1.00      1.00      1.00     12012
weighted avg       1.00      1.00      1.00     12012


Decision Tree:
For Testing data:
Accuracy: 0.789044289044289
Confusion Matrix:
 [[1834  311]
 [ 594 1551]]
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.86      0.80      2145
           1       0.83      0.72      0.77      2145

    accuracy                           0.79      4290
   macro avg       0.79      0.79      0.79      4290
weighted avg       0.79      0.79      0.79      4290



In [39]:
# Random Forest
rf_train_predictions = rf_model.predict(X_train)
print("Random Forest:")
print("For Training data:")
print("Accuracy:", accuracy_score(Y_train, rf_train_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_train, rf_train_predictions))
print("Classification Report:\n", classification_report(Y_train, rf_train_predictions))

rf_test_predictions = rf_model.predict(X_test)
print("Random Forest:")
print("For Testing data:")
print("Accuracy:", accuracy_score(Y_test, rf_test_predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_test, rf_test_predictions))
print("Classification Report:\n", classification_report(Y_test, rf_test_predictions))

Random Forest:
For Training data:
Accuracy: 1.0
Confusion Matrix:
 [[6006    0]
 [   0 6006]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6006
           1       1.00      1.00      1.00      6006

    accuracy                           1.00     12012
   macro avg       1.00      1.00      1.00     12012
weighted avg       1.00      1.00      1.00     12012

Random Forest:
For Testing data:
Accuracy: 0.8282051282051283
Confusion Matrix:
 [[2032  113]
 [ 624 1521]]
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.95      0.85      2145
           1       0.93      0.71      0.80      2145

    accuracy                           0.83      4290
   macro avg       0.85      0.83      0.83      4290
weighted avg       0.85      0.83      0.83      4290

